In [1]:
from pyspark.sql import SparkSession
from pyspark import  SparkContext
import os
import pandas as pd

/tmp/ipykernel_614051/2081191832.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
spark = SparkSession.builder.master("yarn").getOrCreate()

/spark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/16 10:18:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/16 10:18:41 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
import pandas as pd

# Sample data for the Tasks table
tasks_data = {
    'task_id': [1, 2, 3],
    'subtasks_count': [3, 2, 4]
}

# Sample data for the Executed table
executed_data = {
    'task_id': [1, 3, 3, 3, 3],
    'subtask_id': [2, 1, 2, 3, 4]
}

# Convert data to DataFrames
tasks_df = pd.DataFrame(tasks_data)
executed_df = pd.DataFrame(executed_data)

# Display the DataFrames
print("Tasks table:")
print(tasks_df)
print("\nExecuted table:")
print(executed_df)


df_person = spark.createDataFrame(tasks_df)
df_person.createOrReplaceTempView("Tasks")

df_person = spark.createDataFrame(executed_df)
df_person.createOrReplaceTempView("Executed")



Tasks table:
   task_id  subtasks_count
0        1               3
1        2               2
2        3               4

Executed table:
   task_id  subtask_id
0        1           2
1        3           1
2        3           2
3        3           3
4        3           4


In [7]:
query = """
    select
        t.task_id,
        t.subtask_id
    from (
        select
            t.*,
            explode(sequence(1, t.subtasks_count)) as subtask_id
        from Tasks t 
        order by t.task_id, subtask_id asc
    ) as t 
    where (t.task_id, t.subtask_id) not in (
        select task_id, subtask_id from Executed
    )
"""

In [8]:
spark.sql(query).show()

+-------+----------+
|task_id|subtask_id|
+-------+----------+
|      1|         1|
|      1|         3|
|      2|         1|
|      2|         2|
+-------+----------+

